## Automate the approval process for expense reports (with plan and execute architecture )

In [ ]:
import asyncio
from langchain_core.tools import tool
import operator
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from typing import Annotated, List, Tuple, Union, Optional
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver



In [ ]:
#Enhance tools with error handling and input validation


@tool
def validate_expense_report(report_id: str) -> str:
    """Validates an employee's expense report."""
    if not report_id or not isinstance(report_id, str):
        return "Error: Invalid report ID provided"
    try:
        return f"Expense report {report_id} is valid."
    except Exception as e:
        return f"Error validating expense report: {str(e)}"


@tool
def check_policy_compliance(report_id: str) -> str:
    """Checks whether the report complies with company policy"""
    if not report_id or not isinstance(report_id, str):
        return "Error: Invalid report ID provided"
    try:
        return f"Report {report_id} complies with company policy"
    except Exception as e:
        return f"Error checking policy compliance: {str(e)}" 


@tool
def route_to_manager(report_id: str) ->str:
    """Routes the report to the manager for approval."""
    if not report_id or not isinstance(report_id, str):
        return "Error: Invalid report ID provided"
    try:
        return f"Report {report_id} has been routed to the manager"
    except Exception as e:
        return f"Error routing to manager: {str(e)}"     


@tool
def notify_employee(report_id: str, status: str) -> str:
    """Notifies the employee of the report's status."""
    if not report_id or not status:
              return "Error: Invalid report ID or status provided" 
    try:
        return f"Employee notified that report {report_id} is {status}"
    except Exception as e:
        return f"Error notifying employee: {str(e)}"       


tools = [validate_expense_report, check_policy_compliance, route_to_manager, notify_employee]

#Enhanced state management

class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: Optional[str]
    error: Optional[str]

class Plan(BaseModel):
    steps: List[str] = Field(description="Numbered unique steps to follow, in order")

class Response(BaseModel):
    response: str = Field(description="Response to user")

class Act(BaseModel):
    action: Union[Response, Plan] = Field(description="Action to perform")

#Improved system prompts
SYSTEM_PROMPTS = """You are an expense report processing assistant.Your task is to validate and process expense reports
using the available tools.Always extract the report ID from the input and use it consistently across all steps.
available tools:
1. validate_expense_report - Validates the report
2. check_policy_compliance - Checks policy compliance
3. route_to_manager - Routes to manager
4. notify_employee - Notifies the employee
Ensure each steps is completed before moving to the next one .
"""

#Enhance the agent setup
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPTS),
    ("placeholder", "{messages}")
])
llm = ChatOpenAI(model="gpt-40-mini", temperature=0)
agent_executor = create_react_agent(llm, tools, state_modifier= prompt)

#Improved planning step
async def plan_step(state: PlanExecute) -> dict:
    try:
        planner_prompt = ChatPromptTemplate.from_messages([
            ("system", SYSTEM_PROMPTS),
            ("placeholder", "{messages}")
        ])
        planner = planner_prompt | llm.with_structured_output(Plan)
        plan = await planner.ainvoke({"messages": [("user", state["input"])]})
        return {"plan": plan.steps}
    except Exception as e:
        return {"error": f"Planning error: {str(e)}"}
    
#Improved execution step with error handling 
async def execute_step(state: PlanExecute) -> dict:
    try:
            if "error" in state:
                return {"response": f"Workflow failed: {state['error']}"}
            plan = state["plan"]
            if not plan:
                return {"response": "No plan steps available to execute"}
            task = plan[0]
            agent_executor = await agent_executor.ainvoke({"messages":[("user", task)]})
            return {"past_steps": [(task, agent_response["messages"][-1].content)]}
    except Exception as e:
        return {"error": f"Exception error: {str(e)}"}
    
#Enhanced replanning with better error handling 
    